Import PyTorch and other Packages

In [34]:
# import PyTorch
import torch
import torchvision
import os
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'google'

In [2]:
#import other packages
from PIL import Image
from torchvision import transforms

Load ExoNet Image Dataset and the labels from the Dropbox

In [36]:
install google
from google.colab import drive

SyntaxError: invalid syntax (Temp/ipykernel_4496/1006074814.py, line 1)

In [3]:
#Load Images from Dropbox
import requests
from requests.auth import HTTPBasicAuth
import shutil


images_url = 'https://www.dropbox.com/s/30wjsbnrv039df5/ExoNet_Test.zip?dl=1'
db_username = 'agzkurbis@uwaterloo.ca'
db_password = 'Temppassword123!'

downloaded_file = requests.get(images_url, auth=HTTPBasicAuth(db_username, db_password))

# dest_file = open('/Users/aj/Desktop/test.jpg', 'w+')

# dest_file.write(downloaded_file.content)

In [30]:
# ! copy "D:\\UofT_Project\\CV_Research_Code_02\\Dataset"

In [16]:
import shutil
filename = 'D:\\UofT_Project\\CV_Research_Code_02\\ExoNet_Test.zip'
extract_dir = "D:\\UofT_Project\\CV_Research_Code_02\\Dataset"
shutil.unpack_archive(filename, extract_dir)

In [14]:
#Load ExoNet labels from Dropbox
# https://www.dropbox.com/s/ksodxkpxzcmtspu/Labels.csv?dl=0

labels_url = 'https://www.dropbox.com/s/ksodxkpxzcmtspu/Labels.csv?dl=1'
db_username = 'agzkurbis@uwaterloo.ca'
db_password = 'Temppassword123!'

downloaded_file = requests.get(labels_url, auth=HTTPBasicAuth(db_username, db_password))

Sort Images into training/validation/testing 
this split should be as follows:
- 89.5% Training
- 3.5% Validation 
- 7% Testing 

In [32]:
#Creating a definition for randomized dataset creation

def train_validate_test_split(df, train_percent=.895, validate_percent=.035, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [33]:
#Randomize Images and split into Training/Validation/Testing

#load directory df
IMG_DIR = "D:\\UofT_Project\\CV_Research_Code_02\\Dataset"

dataset = []

for fold in os.listdir(IMG_DIR):
    for filename in os.listdir(f'{IMG_DIR}/{fold}'):
        dataset.append((f'{fold}/{filename}', fold))

df = pd.DataFrame(dataset, columns=['filename', 'category'])


df_train, df_validation, df_test = train_validate_test_split(df, train_percent=.895, validate_percent=.035)

df_train['set'] = 'train'
df_validation['set'] = 'validation'
df_test['set'] = 'test'

#what does this do? Needs to be updated to include validation labels as well 
df = df_train.append(df_validation)
df = df_train.append(df_test)
df.to_csv('dataset.csv', index=False)
df.head()

C:\Users\akurb\anaconda3\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akurb\anaconda3\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akurb\anaconda3\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

,filename,category,set
983,LG-T-IS/['IMG_01_1'] frame 52116.jpg,LG-T-IS,train
676,LG-S/['IMG_01_1'] frame 2070.jpg,LG-S,train
400,IS-T-DW/['IMG_01_1'] frame 16614.jpg,IS-T-DW,train
1096,LG-T-O/['IMG_01_1'] frame 6774.jpg,LG-T-O,train
398,IS-S/['IMG_01_1'] frame 46788.jpg,IS-S,train


Do we need to reset the index and drop images? (skip this for now as discussed in previous meeting)

In [19]:
#reset index and drop images
train_df = df[df.set == 'train'].reset_index(drop=True)
validate_df = df[df.set == 'validation'].reset_index(drop=True)
test_df = df[df.set == 'test'].reset_index(drop=True)

Resize/formate images?

In [20]:
#define dementions of the images we are using (this value will need to be adjusted)
img_width, img_height = 224, 224
IMAGE_SIZE = (img_width, img_height)

#formating 
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

NameError: name 'K' is not defined

Initializing MobileNetV2 Model

In [ ]:
#Initialize MobileNetV2 Model

model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False)
model.eval()

In [ ]:
#Defining Parameters
#needs to be updated for torch (see thesis code)
model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 1), 
    alpha=0.5,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=max,
    classes=4
)

output = model.output

#Look into parameter adjustment using validation data

In [ ]:
#Select optimizer, loss type, and metrics (see thesis)
#compile the model

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Do we need this?
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
#save the Model Weights for MobileNetV2 to "______"

#determine the best location for this? maybe use OneDrive since we are using Azure
#I looked into dropbox and this seems difficult to save directly to dropbox. Might be better through a google drive, onedrive, ext.

#research/update parameters here
callbacks_list = [
    ModelCheckpoint('/content/gdrive/My Drive/552-project/service_weights.h5', 
                    monitor='acc', verbose=1, save_best_only=True, mode='max'),
        ReduceLROnPlateau(monitor='acc', 
                                            patience=1, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
]

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    IMG_DIR, 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    color_mode = 'grayscale',
    class_mode='categorical',
    batch_size = BATCH_SIZE
)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    IMG_DIR,
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    color_mode = 'grayscale',
    class_mode='categorical',
    shuffle=False,
    batch_size = BATCH_SIZE
)

test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    IMG_DIR,
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    color_mode = 'grayscale',
    class_mode='categorical',
    shuffle=False,
    batch_size = BATCH_SIZE
)

#for initial train/validation
history = model.fit_generator(
    generator=train_generator, 
    steps_per_epoch=9,
    verbose=1,
    epochs=20,
    validation_data=validation_generator,
    )

In [ ]:
output = model.output
print(output)

Creating the Confusion Matrix and Classification Report

In [ ]:
#update
num_of_test_samples= test_datagen.length


Y_pred = model.predict_generator(validation_generator, num_of_test_samples // BATCH_SIZE+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['LG', 'LG_IS', 'IS','IS_LG', 'DS_LG', 'LS_DS']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [ ]:
#print the confusion matrix
print(confusion_matrix(validate_df.category, validate_df.pred))

In [ ]:
#creating a plot for the accuracy vs epoch 

import matplotlib.pyplot as plt

acc = history.history['acc']
# val_acc = history.history['val_acc']
loss = history.history['loss']
# val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.subplot(1, 2, 1)
plt.plot(model.history.history['loss'])
plt.ylabel('training loss (error)')
plt.xlabel('# of epochs')
plt.title('training')

plt.subplot(1, 2, 2)
plt.plot(model.history.history['acc'])
plt.ylabel('training accuracy')
plt.xlabel('# of epochs')
plt.title('training')
plt.tight_layout()

In [ ]:
#Code to save the model
from google.colab import drive
drive.mount('/content/gdrive')

#update location
model.save('/content/gdrive/My Drive/CV_Research/model_new.h5')